<a href="https://colab.research.google.com/github/narejoneduet/A-University-Microgrid-Model/blob/main/Dynamic_Energy_Pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

# Define fictitious data
electricity_generation_cost = np.array([10, 20, 30, 40, 50])
transmission_cost = np.array([1, 2, 3, 4, 5])
consumer_demand = np.array([100, 200, 300, 400, 500])
total_supply_capacity = 1000
min_price = 0.1
max_price = 1

# Define the objective function
def objective_function(price):
    total_cost = electricity_generation_cost[price] + transmission_cost[price]
    return total_cost

# Define constraints
def demand_constraint(price):
    total_demand = consumer_demand[price]
    return total_demand <= total_supply_capacity

def supply_constraint(price):
    total_generation = electricity_generation_cost[price]
    return total_generation <= total_supply_capacity

def pricing_constraint(price):
    return min_price <= price <= max_price

def switching_constraint(scheme):
    return scheme in {'RTP', 'TOU', 'HEU'}

# Set up the optimization problem
n_consumers = len(consumer_demand)
bounds = [(min_price, max_price)] * n_consumers
constraints = [demand_constraint, supply_constraint, pricing_constraint]

# Solve the optimization problem
result = linprog([1] * n_consumers, bounds=bounds, constraints=constraints, method='simplex')

# Get the optimal dynamic energy pricing scheme
optimal_scheme = ['RTP' if x < 0.33 else ('TOU' if x < 0.67 else 'HEU') for x in result.x]

# Print the optimal dynamic energy pricing scheme
for consumer, scheme in enumerate(optimal_scheme):
    print(f"Consumer {consumer + 1}: {scheme}")


In [ ]:
import numpy as np
from scipy.optimize import linprog

# Define fictitious data
electricity_generation_cost = np.array([10, 20, 30, 40, 50])
transmission_cost = np.array([1, 2, 3, 4, 5])
consumer_demand = np.array([100, 200, 300, 400, 500])
total_supply_capacity = 1000
min_price = 0.1
max_price = 1

# Number of consumers
n_consumers = len(consumer_demand)

# Define the coefficients for the objective function
c = np.concatenate((electricity_generation_cost, transmission_cost))

# Define the inequality constraints (A_ub and b_ub)
A_ub = np.zeros((2 * n_consumers, 2 * n_consumers))
b_ub = np.zeros(2 * n_consumers)

for i in range(n_consumers):
    A_ub[i, i] = 1  # Demand constraint
    A_ub[n_consumers + i, i] = 1  # Supply constraint
    b_ub[i] = consumer_demand[i]
    b_ub[n_consumers + i] = total_supply_capacity

# Define bounds for prices
bounds = [(min_price, max_price) for _ in range(n_consumers * 2)]

# Solve the optimization problem
result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='simplex')

# Get the optimal prices
optimal_prices = result.x[:n_consumers]

# Print the optimal dynamic energy pricing scheme
for consumer, price in enumerate(optimal_prices):
    if min_price <= price < 0.33:
        scheme = 'RTP'
    elif 0.33 <= price < 0.67:
        scheme = 'TOU'
    else:
        scheme = 'HEU'
    print(f"Consumer {consumer + 1}: {scheme}")


In [ ]:
import numpy as np
from scipy.optimize import linprog
import random

# Define fictitious data
electricity_generation_cost = np.array([10, 20, 30, 40, 50])
transmission_cost = np.array([1, 2, 3, 4, 5])
consumer_demand = np.array([100, 200, 300, 400, 500])
total_supply_capacity = 1000
min_price = 0.1
max_price = 1

# Define consumer energy efficiency values (random values for illustration)
energy_efficiency = np.array([0.7, 0.8, 0.6, 0.9, 0.5])

# Define the consumers' types
consumer_types = ['TOU' for _ in range(3)] + ['RTP' for _ in range(2)]
random.shuffle(consumer_types)

# Number of consumers
n_consumers = len(consumer_demand)

# Define the coefficients for the objective function
c = np.concatenate((electricity_generation_cost, transmission_cost))

# Define the inequality constraints (A_ub and b_ub)
A_ub = np.zeros((2 * n_consumers, 2 * n_consumers))
b_ub = np.zeros(2 * n_consumers)

for i in range(n_consumers):
    A_ub[i, i] = 1  # Demand constraint
    A_ub[n_consumers + i, i] = 1  # Supply constraint
    b_ub[i] = consumer_demand[i]
    b_ub[n_consumers + i] = total_supply_capacity

# Define bounds for prices
bounds = [(min_price, max_price) for _ in range(n_consumers * 2)]

# Solve the optimization problem
result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='simplex')

# Get the optimal prices
optimal_prices = result.x[:n_consumers]

# Print the optimal dynamic energy pricing scheme and energy efficiency for each consumer
for consumer, price, efficiency, consumer_type in zip(range(1, n_consumers + 1), optimal_prices, energy_efficiency, consumer_types):
    if min_price <= price < 0.33:
        scheme = 'RTP'
    elif 0.33 <= price < 0.67:
        scheme = 'TOU'
    else:
        scheme = 'HEU'
    print(f"Consumer {consumer}: Pricing Scheme: {scheme}, Energy Efficiency: {efficiency:.2f}")
